<a href="https://colab.research.google.com/github/sunflowerseed17/PythonDSProject/blob/main/Feature_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Using TfidfVectorizer from Scikit-learn

## TODO: N-Grams

### TODO: Unigrams and Bigrams used to capture context (unigrams = single words, bigrams = pairs of words)

### TODO: Using TF-IDF (Term Frequency-Inverse Document Frequency) to weight the unigrams and bigrams depending on importance

In [ ]:
from scikit-learn import tfidfvectorizer

## TODO: LIWC (Linguistic Inquiry and Word Count)

### Use LIWC as a feature extractor (using the LIWC2015 dictionary)

Extract 68 features from LIWC, focusing of categories of Psychological Processes, Linguistic Dimensions, Personal Concerns.

## TODO: Topic Modelling using LDA (Latent Dirichlet Allocation)

### Use the Mallet toolkit or Gensim library in Python to apply LDA

In [ ]:
import mallet
import gensim